In [1]:
import math
import pandas as pd

from absl import app
from absl import flags

import tensorflow as tf
import functools
from tqdm import tqdm
import os
from pathlib import Path
from model.model import *
from utils import preprocessing

In [2]:
train_file_pattern = "/home/ubuntu/Dacon/jin/NIA/tfrecords/train*"
validation_file_pattern = "/home/ubuntu/Dacon/jin/NIA/tfrecords/validation*"
global_batch_size = 128
image_size = 224
augmentation = False
seed = 0

In [3]:
EPOCHS = 10
batch_size = 32
image_size = 224
STEPS_PER_TPU_CALL = 1
learning_rate = 1e-5  # should be smaller than training on single GPU
feature_size = 2048  # Embedding size before the output layer
save_interval = 1000

# ArcFace params
margin = 0.1  # DELG used 0.1, original ArcFace paper used 0.5. When margin is 0, it should be the same as doing a normal softmax but with embedding and weight normalised.
logit_scale = int(math.sqrt(feature_size))

# GeM params
gem_p = 3.
train_p = False  # whether to learn gem_p or not

data_dir = "/home/ubuntu/Dacon/jin/NIA"

In [4]:
data_dir = "/home/ubuntu/Dacon/jin/NIA"
training_csv_path = os.path.join(data_dir, "train.csv")
train_csv = pd.read_csv(str(training_csv_path))
num_samples = len(train_csv["id"].tolist())
unique_landmark_ids = train_csv["landmark_id"].unique().tolist()
unique_landmark_ids = tf.convert_to_tensor(unique_landmark_ids, dtype=tf.int64)

In [ ]:
tra

In [5]:
train_dataset = preprocessing.CreateDataset(
      file_pattern=train_file_pattern,
      batch_size=global_batch_size,
      image_size=image_size,
      unique_landmark_ids = unique_landmark_ids,
      seed=seed)

In [7]:
validation_dataset = preprocessing.CreateDataset(
      file_pattern=validation_file_pattern,
      batch_size=global_batch_size,
      image_size=image_size,
      unique_landmark_ids = unique_landmark_ids,
      seed=seed)

In [7]:
EPOCHS = 10
batch_size = 32
image_size = 224
learning_rate = 1e-4  # should be smaller than training on single GPU
feature_size = 2048  # Embedding size before the output layer
save_interval = 1000

# ArcFace params
margin = 0.1  # DELG used 0.1, original ArcFace paper used 0.5. When margin is 0, it should be the same as doing a normal softmax but with embedding and weight normalised.
logit_scale = int(math.sqrt(feature_size))

# GeM params
gem_p = 3.
train_p = False  # whether to learn gem_p or not

data_dir = "/home/ubuntu/Dacon/jin/NIA"

training_csv_path = os.path.join(data_dir, "train.csv")
train_csv = pd.read_csv(str(training_csv_path))
num_samples = len(train_csv["id"].tolist())
unique_landmark_ids = train_csv["landmark_id"].unique().tolist()
unique_landmark_ids = tf.convert_to_tensor(unique_landmark_ids, dtype=tf.int64)

In [8]:
input = tf.keras.layers.Input(shape=(224, 224, 3))
label = tf.keras.layers.Input(shape=(len(unique_landmark_ids),))
x = tf.keras.applications.ResNet101(input_shape=input.shape[1:],
                                               include_top=False,
                                               weights='imagenet')
x = x(input)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(2048)(x)
output = ArcFaceLayer(len(unique_landmark_ids), margin, logit_scale)([x, label])

In [9]:
model = tf.keras.Model([input, label], output)

In [7]:
model = DelfArcFaceModel(
            input_shape=(image_size, image_size, 3), n_classes=len(unique_landmark_ids), margin=margin, logit_scale=logit_scale,
            p=gem_p, train_p=train_p, feature_size=feature_size
        )

In [10]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

In [11]:
# 파일 이름에 에포크 번호를 포함시킵니다(`str.format` 포맷)
checkpoint_path = "./checkpoint/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 다섯 번째 에포크마다 가중치를 저장하기 위한 콜백을 만듭니다
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    period=10)

In [12]:
for 
model.fit(train_dataset,
          ,
    epochs=200,
    steps_per_epoch=100,
    callbacks=[cp_callback])

ValueError: `y` argument is not supported when using dataset as input.

In [ ]:
DelfArcFaceModel(
            input_shape=(image_size, image_size, 3), n_classes=len(unique_landmark_ids), margin=margin, logit_scale=logit_scale,
            p=gem_p, train_p=train_p, feature_size=feature_size
        )

In [27]:
DelfArcFaceModel.fit()

TypeError: _method_wrapper() missing 1 required positional argument: 'self'

In [26]:
train_dataset.split

AttributeError: 'BatchDataset' object has no attribute 'split'